# Projeto 13: Rede Neural Convolucional MNIST Aumento de Imagens

## Etapa 1: Importação das bibliotecas

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torchvision import datasets
import torchvision.transforms as transforms
torch.__version__

'1.13.1+cu116'

## Etapa 2: Base de dados

In [2]:
torch.manual_seed(123)

In [3]:
transform_train = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(degrees = 7, 
                                                              translate = (0, 0.07), # horizontal and vertical shifts
                                                              shear = 7,
                                                              scale = (1, 1.2) # zoom range
                                                              ),
                                      transforms.ToTensor()
                                      ])

In [4]:
transform_test = transforms.ToTensor()

In [5]:
train = datasets.MNIST(root = '.', train = True, download = True, transform = transform_train)
test = datasets.MNIST(root = '.', train = False, download = True, transform = transform_test)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [6]:
train_loader = torch.utils.data.DataLoader(train, batch_size = 128)
test_loader = torch.utils.data.DataLoader(test, batch_size = 128)

## Etapa 3: Construção do modelo

In [7]:
class classificador(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = (3, 3))
        self.activation = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size = (2, 2))
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(in_features = 32*13*13, out_features = 128)
        self.output = nn.Linear(128, 10)
        
    def forward(self, X):
        X = self.pool(self.activation(self.conv(X)))
        X = self.flatten(X)
        X = self.activation(self.linear(X))
        X = self.output(X)

        return X

In [8]:
net = classificador()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [10]:
device

device(type='cuda')

In [11]:
net.to(device)

classificador(
  (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (activation): ReLU()
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=5408, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=10, bias=True)
)

## Etapa 4: Treinamento do modelo

In [12]:
def training_loop(loader, epoch):

    running_loss = 0.
    running_accuracy = 0.
    
    for i, data in enumerate(loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()        

        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()

        running_loss += loss.item()
        
        ps = F.softmax(outputs)
        top_p, top_class = ps.topk(k = 1, dim = 1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy = torch.mean(equals.type(torch.float))
        
        running_accuracy += accuracy
                
        # Imprimindo os dados referentes a este loop
        print('\rÉpoca {:3d} - Loop {:3d} de {:3d}: perda {:03.2f} - precisão {:03.2f}'.format(epoch + 1, i + 1, len(loader), loss, accuracy), end = '\r')
        
    # Imprimindo os dados referentes a esta época
    
    print('\rÉPOCA {:3d} FINALIZADA: perda {:.5f} - precisão {:.5f}'.format(epoch+1, running_loss/len(loader), running_accuracy/len(loader)))

In [13]:
for epoch in range(5):
    print('Treinando...')
    training_loop(train_loader, epoch)

    net.eval()
    print('Validando...')
    training_loop(test_loader, epoch)

    net.train()

Treinando...


<ipython-input-12-e826541fba46>:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ps = F.softmax(outputs)


ÉPOCA   1 FINALIZADA: perda 0.61231 - precisão 0.80531
Validando...
ÉPOCA   1 FINALIZADA: perda 0.18292 - precisão 0.94432
Treinando...
ÉPOCA   2 FINALIZADA: perda 0.26358 - precisão 0.91788
Validando...
ÉPOCA   2 FINALIZADA: perda 0.10963 - precisão 0.96519
Treinando...
ÉPOCA   3 FINALIZADA: perda 0.19588 - precisão 0.93940
Validando...
ÉPOCA   3 FINALIZADA: perda 0.08037 - precisão 0.97360
Treinando...
ÉPOCA   4 FINALIZADA: perda 0.16190 - precisão 0.94973
Validando...
ÉPOCA   4 FINALIZADA: perda 0.05647 - precisão 0.98200
Treinando...
ÉPOCA   5 FINALIZADA: perda 0.14001 - precisão 0.95643
Validando...
ÉPOCA   5 FINALIZADA: perda 0.04621 - precisão 0.98705
